# MNIST softmax

* MNIST data를 가지고 softmax classifier를 만들어보자.
  * [참고: TensorFlow.org](https://www.tensorflow.org/get_started/mnist/beginners)
  * [소스: mnist_softmax.py](https://github.com/tensorflow/tensorflow/blob/r1.4/tensorflow/examples/tutorials/mnist/mnist_softmax.py)

### Import modules

In [ ]:
"""A very simple MNIST classifier.
See extensive documentation at
https://www.tensorflow.org/get_started/mnist/beginners
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))

tf.set_random_seed(219)

### Import data

In [ ]:
data_dir = '../mnist'
mnist = input_data.read_data_sets(data_dir, one_hot=True)

### Show the MNIST

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

index = 100
print("label = ", np.argmax(mnist.train.labels[index]))
plt.imshow(mnist.train.images[index].reshape(28, 28), cmap='gray')
plt.show()

### Create the model

In [ ]:
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.matmul(x, W) + b

### Define loss and optimizer

* [`tf.nn.softmax_cross_entropy_with_logits`](https://www.tensorflow.org/api_docs/python/tf/nn/softmax_cross_entropy_with_logits)

In [ ]:
y_ = tf.placeholder(tf.float32, [None, 10])
# The raw formulation of cross-entropy,
#
#   tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.nn.softmax(y)),
#                                 reduction_indices=[1]))
#
# can be numerically unstable.
#
# So here we use tf.nn.softmax_cross_entropy_with_logits on the raw
# outputs of 'y', and then average across the batch.
cross_entropy = tf.reduce_mean(
                    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

### tf.InteractiveSession() and train

In [ ]:
sess = tf.InteractiveSession(config=sess_config)
tf.global_variables_initializer().run()
# Train
max_step = 100
for step in range(max_step+1):
  batch_xs, batch_ys = mnist.train.next_batch(32)
  _, loss = sess.run([train_step, cross_entropy], feed_dict={x: batch_xs, y_: batch_ys})
  if step % 10 == 0:
    print("step: %d, loss: %g" % (step, loss))

### Test trained model

* test accuracy: 0.8731

In [ ]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                    y_: mnist.test.labels}))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

test_batch_size = 16
batch_xs, batch_ys = mnist.test.next_batch(test_batch_size)
y_pred = sess.run(y, feed_dict={x: batch_xs})

fig = plt.figure(figsize=(16, 10))
for i, (px, py) in enumerate(zip(batch_xs, y_pred)):
  p = fig.add_subplot(4, 8, i+1)
  p.set_title("y_pred: {}".format(np.argmax(py)))
  p.imshow(px.reshape(28, 28), cmap='gray')
  p.axis('off')

## 직접 실습

* 여러가지 hyper-parameter들을 바꿔가면서 accuracy를 높혀보자